[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davemlz/eemont/blob/master/docs/tutorials/005-EVI-with-Overloaded-Operators-Sentinel-2.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/davemlz/eemont/blob/master/docs/tutorials/005-EVI-with-Overloaded-Operators-Sentinel-2.ipynb)
[![Open in Planetary Computer](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/davemlz/eemont&urlpath=lab/tree/eemont/docs/tutorials/005-EVI-with-Overloaded-Operators-Sentinel-2.ipynb&branch=master)

# Using Overloaded Operators to Compute the EVI (Sentinel-2)

_Tutorial created by **David Montero Loaiza**_: [GitHub](https://github.com/davemlz) | [Twitter](https://twitter.com/dmlmont)

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Conda-forge: [https://anaconda.org/conda-forge/eemont](https://anaconda.org/conda-forge/eemont)
- Documentation: [https://eemont.readthedocs.io/](https://eemont.readthedocs.io/)
- More tutorials: [https://github.com/davemlz/eemont/tree/master/docs/tutorials](https://github.com/davemlz/eemont/tree/master/docs/tutorials)

## Let's start!

If required, please uncomment:

In [1]:
!pip install eemont
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=fd5b295c4ee0e3d36c0fd66fe5f3ec74aa1678efd2741d3d21f5f21b7ad67fb9
  Stored in directory: /root/.cache/pip/wheels/5f/c3/91/8921b158625d57be320249a7b4ed5e6a0bee5bf8395d8591f6
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236753 sha256=134da5607c788cfdfef2c07d2014cd5295e453bb7fa83562d75b3d6c12c11e8d
  Stored in directory: /root/.cache/pip/wheels/04/f2/5f/1ab40801f6334408e009cf01dca51b0359f3356329ee0b464b
Successfully built eemont ee_extra
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:00:00


Import the required packages.

In [2]:
import ee, eemont, datetime, geemap

In [5]:
ee.Authenticate()
ee.Initialize(project='gee-wildfire-ai')

Authenticate and Initialize Earth Engine and geemap.

In [7]:
Map = geemap.Map()

Point of interest.

In [8]:
point = ee.Geometry.Point([-76.0269,2.92846])

Get, filter, mask clouds and scale the image collection.

In [9]:
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
      .maskClouds()
      .scaleAndOffset()
      .spectralIndices('EVI')) # Let's compute the EVI using the index() method for comparison

Let's select the required bands for EVI:

In [10]:
N = S2.select('B8')
R = S2.select('B4')
B = S2.select('B2')

## Overloaded Operators

`eemont` has overloaded the binary operators, rich comparisons and unary operators in the following list for the `ee.Image` class:

(+, -, \*\, /, //, %, \**\, <<, >>, &, |, <, <=, ==, !=, >, >=, -, ~)

Therefore, you can now use them for image operations!

In [11]:
EVI = 2.5 * (N - R) / (N + 6.0 * R - 7.5 * B + 1.0)

## Visualization

Let's define the EVI visualization parameters:

In [12]:
visEVI = {
    'min':0,
    'max':1,
    'palette':[
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000',
        '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301'
    ]
}

And also the RGB visualization parameters:

In [13]:
visRGB = {
    'min':0,
    'max':0.3,
    'bands':['B4', 'B3', 'B2']
}

Use `geemap` to display results:

In [15]:
Map.addLayer(S2,visRGB,'RGB')
Map.addLayer(EVI,visEVI,'EVI With Overloaded Operators')
Map.addLayer(S2.select('EVI'),visEVI,'EVI With index() Method')
# Map.add_colorbar(visEVI['palette'], caption = 'EVI')
Map.centerObject(point,9)
Map

Map(center=[2.92846, -76.0269], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…